In [132]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as exu, estimator_util as esu
import importlib
from typing import List, Dict, Any
import os

In [4]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [213]:
# Usage example
dataset = du.DatasetLoader(EXT_DATA_DIR)
train_data = dataset.load_all_scenes()


In [219]:
importlib.reload(du)

<module 'utils.data_util' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/data_util.py'>

In [199]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [217]:
default_sift_config = exu.SIFTConfig()
extractor_config = exu.FeatureExtractorConfig(default_sift_config)
extractor = exu.FeatureExtractor(config=extractor_config)

In [221]:
# function for preprocessing all the dataset. only needs the path to the dataset and run once
for scene in train_data:
    scene_data = train_data[scene]
    for img in scene_data.images_dir.iterdir():
        image_name = img.name.replace(img.suffix, '')
        if scene_data.image_data is None:
                scene_data.image_data = {}

        preprocessed_img = preprocessor.process_image(img)
        image_features = extractor.extract_features(preprocessed_img)
        scene_data.image_data[image_name] = du.ImageData(image_name, img, preprocessed_img, image_features)

In [69]:
i1f = extractor.extract_features(img1)
i2f = extractor.extract_features(img2)

In [218]:
i1f

ImageFeature(keypoints=(< cv2.KeyPoint 0x10d12ab50>, < cv2.KeyPoint 0x10d12e9a0>, < cv2.KeyPoint 0x10ce1f000>, < cv2.KeyPoint 0x10ce1ff00>, < cv2.KeyPoint 0x10fff2220>, < cv2.KeyPoint 0x10fff3360>, < cv2.KeyPoint 0x10fff3000>, < cv2.KeyPoint 0x10fff0db0>, < cv2.KeyPoint 0x110049320>, < cv2.KeyPoint 0x1100481e0>, < cv2.KeyPoint 0x11004b930>, < cv2.KeyPoint 0x11004ab80>, < cv2.KeyPoint 0x110049860>, < cv2.KeyPoint 0x11004a460>, < cv2.KeyPoint 0x110048d20>, < cv2.KeyPoint 0x110048810>, < cv2.KeyPoint 0x110048c90>, < cv2.KeyPoint 0x110048210>, < cv2.KeyPoint 0x110049440>, < cv2.KeyPoint 0x11004a550>, < cv2.KeyPoint 0x11004a4f0>, < cv2.KeyPoint 0x110048b70>, < cv2.KeyPoint 0x11004a2e0>, < cv2.KeyPoint 0x1100491d0>, < cv2.KeyPoint 0x110048630>, < cv2.KeyPoint 0x11004a400>, < cv2.KeyPoint 0x110049fb0>, < cv2.KeyPoint 0x1100493b0>, < cv2.KeyPoint 0x110048a80>, < cv2.KeyPoint 0x110048780>, < cv2.KeyPoint 0x110048420>, < cv2.KeyPoint 0x110048300>, < cv2.KeyPoint 0x110048030>, < cv2.KeyPoint 0x11

In [124]:
bf_config = esu.BFMatcherConfig(knn=True)
matcher_config = esu.MatcherConfig(bf_config)

In [125]:
matcher = esu.FeatureMatcher(matcher_config)

In [126]:
matches = matcher.match_features(i1f, i2f)

In [127]:
valid, kp1, kp2 = matcher.filter_lowe_matches(matches, i1f, i2f)

In [141]:
type(kp1[0])



cv2.KeyPoint

In [143]:
es_alg_conf = esu.RANSACConfig()
es_conf = esu.EstimatorConfig('RANSAC', es_alg_conf)


In [144]:
fe = esu.FundamentalMatrixEstimator(kp1, kp2, es_conf)

In [145]:
fe.estimate()

array([[ 2.19455369e-06, -1.28450711e-04,  5.75317978e-02],
       [ 1.10165394e-04,  6.54828648e-07, -2.44261228e-02],
       [-6.91245930e-02,  3.13106405e-02,  1.00000000e+00]])

In [149]:
inl = fe.get_inliers()

In [ ]:
# Convert inliers to a set of tuples for easy comparison
inliers_set = set(map(tuple, inl[0]))
print(f"Found {len(inliers_set)} inliers from {len(inl[0])} inliers")

# Find outliers by checking which keypoints in kp1 are not in inliers
outliers = [kp for kp in kp1 if (kp.pt[0], kp.pt[1]) not in inliers_set]

print(f"Inliers count: {len(inliers_set)} out of {len(kp1)}")
print("Total outliers found: ")
print(len(set(outliers)))

Found 24 inliers from 28 INLIERS
Inliers count: 24 out of 59
Total outliers found: 
31
